# Make spectral libraries

In [1]:
import sys, os
sys.path.append('/Users/simon/git/vimms')
sys.path.insert(0,'/Users/simon/git/mass-spec-utils/')
from vimms.Common import save_obj
from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
library_cache = '/Users/simon/clms_er/library_cache'

## Massbank

In [8]:
from mass_spec_utils.library_matching.spec_libraries import MassBankLibrary

Path to the local version of the massbank repo

In [5]:
massbank_data_path = '/Users/simon/git/MassBank-Data/' # final slash is important!

In [ ]:
mb = MassBankLibrary(mb_dir=massbank_data_path, polarity='POSITIVE')

In [ ]:
save_obj(mb, os.path.join(library_cache, 'massbank_pos.p'))

In [6]:
mb = MassBankLibrary(mb_dir=massbank_data_path, polarity='NEGATIVE')
save_obj(mb, os.path.join(library_cache, 'massbank_neg.p'))

Loading records from  /Users/simon/git/MassBank-Data/Athens_Univ/
	 Loaded 5252 new records
Loading records from  /Users/simon/git/MassBank-Data/MetaboLights/
	 Loaded 58 new records
Loading records from  /Users/simon/git/MassBank-Data/MPI_for_Chemical_Ecology/
	 Loaded 691 new records
Loading records from  /Users/simon/git/MassBank-Data/JEOL_Ltd/
	 Loaded 45 new records
Loading records from  /Users/simon/git/MassBank-Data/GL_Sciences_Inc/
	 Loaded 174 new records
Loading records from  /Users/simon/git/MassBank-Data/Env_Anal_Chem_U_Tuebingen/
	 Loaded 128 new records
Loading records from  /Users/simon/git/MassBank-Data/RIKEN_ReSpect/
	 Loaded 4642 new records
Loading records from  /Users/simon/git/MassBank-Data/Boise_State_Univ/
	 Loaded 4 new records
Loading records from  /Users/simon/git/MassBank-Data/LCSB/
	 Loaded 7299 new records
Loading records from  /Users/simon/git/MassBank-Data/PFOS_research_group/
	 Loaded 413 new records
Loading records from  /Users/simon/git/MassBank-Data/E

2020-10-07 12:34:05.501 | INFO     | vimms.Common:save_obj:166 - Saving <class 'mass_spec_utils.library_matching.spec_libraries.MassBankLibrary'> to /Users/simon/clms_er/library_cache/massbank_neg.p


In [9]:
mb = MassBankLibrary(mb_dir=massbank_data_path, polarity='all')
save_obj(mb, os.path.join(library_cache, 'massbank_all.p'))

Loading records from  /Users/simon/git/MassBank-Data/Athens_Univ/
	 Loaded 5252 new records
Loading records from  /Users/simon/git/MassBank-Data/MetaboLights/
	 Loaded 58 new records
Loading records from  /Users/simon/git/MassBank-Data/MPI_for_Chemical_Ecology/
	 Loaded 691 new records
Loading records from  /Users/simon/git/MassBank-Data/JEOL_Ltd/
	 Loaded 45 new records
Loading records from  /Users/simon/git/MassBank-Data/GL_Sciences_Inc/
	 Loaded 174 new records
Loading records from  /Users/simon/git/MassBank-Data/Env_Anal_Chem_U_Tuebingen/
	 Loaded 128 new records
Loading records from  /Users/simon/git/MassBank-Data/RIKEN_ReSpect/
	 Loaded 4642 new records
Loading records from  /Users/simon/git/MassBank-Data/Boise_State_Univ/
	 Loaded 4 new records
Loading records from  /Users/simon/git/MassBank-Data/LCSB/
	 Loaded 7299 new records
Loading records from  /Users/simon/git/MassBank-Data/PFOS_research_group/
	 Loaded 413 new records
Loading records from  /Users/simon/git/MassBank-Data/E

2020-10-07 12:39:34.587 | INFO     | vimms.Common:save_obj:166 - Saving <class 'mass_spec_utils.library_matching.spec_libraries.MassBankLibrary'> to /Users/simon/clms_er/library_cache/massbank_all.p


## GNPS

Using Florian's file, because it has inchikeys

In [ ]:
json_file = '/Users/simon/Downloads/gnps_positive_ionmode_cleaned_by_matchms_and_lookups.json'
import json
with open(json_file,'r') as f:
    payload = json.loads(f.read())

In [ ]:
from mass_spec_utils.library_matching.spectrum import SpectralRecord
neg_intensities = []
def json_to_spectrum(json_dat):
    precursor_mz = json_dat['precursor_mz']
    original_file = json_file
    spectrum_id = json_dat['spectrum_id']
    inchikey = json_dat['inchikey_smiles']
    peaks = json_dat['peaks_json']
    metadata = {}
    for k,v in json_dat.items():
        if not k == 'peaks':
            metadata[k] = v
    mz,i = zip(*peaks)
    if min(i) < 0:
        neg_intensities.append(spectrum_id)
        return None
    else:
        new_spectrum = SpectralRecord(precursor_mz, peaks, metadata, original_file, spectrum_id)
        return new_spectrum

records = {}
for jd in tqdm(payload):
    new_spec = json_to_spectrum(jd)
    if new_spec is not None:
        records[new_spec.spectrum_id] = new_spec

In [ ]:
def filter_min_peaks(spectrum, min_n_peaks=10):
    n_peaks = len(spectrum.peaks)
    if n_peaks < min_n_peaks:
        return None
    else:
        return spectrum
def filter_rel_intensity(spectrum, min_rel=0.01, max_rel=1.):
    pp = spectrum.peaks
    mz,i = zip(*pp)
    max_i = max(i)
    new_pp = []
    for p in pp:
        ri = p[1]/max_i
        if ri <= max_rel and ri >= min_rel:
            new_pp.append(p)
    spectrum.peaks = new_pp
    return spectrum

In [ ]:
new_records = {}
for sid in tqdm(records.keys()):
    spec = records[sid]
    ss = filter_min_peaks(spec)
    if ss is not None:
        new_records[sid] = ss
    else:
        continue
    ss = filter_rel_intensity(ss)
    new_records[sid] = ss

In [ ]:
for sid, ss in new_records.items():
    ss.metadata['inchikey'] = ss.metadata['inchikey_smiles']

In [ ]:
from mass_spec_utils.library_matching.spec_libraries import SpectralLibrary
sl = SpectralLibrary()
sl.records = new_records
sl.sorted_record_list = sl._dic2list()


In [ ]:
save_obj(sl, os.path.join(library_cache,'gnps.p'))